# 作業 : (Kaggle)鐵達尼生存預測
***
https://www.kaggle.com/c/titanic

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(3), object(5)
memory usage: 69.7+ KB


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
df.select_dtypes(include=["object"]).describe()

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Doling, Miss. Elsie",male,CA. 2343,C23 C25 C27,S
freq,1,577,7,4,644


In [4]:
# 先補闕漏值為一類
df['Cabin'] = df['Cabin'].fillna('No')

In [5]:
# 用Cabin艙等groupby,對Fare做統計值處理
df_FareMax = df.groupby(['Cabin'])['Fare'].max().reset_index()
df_FareMin = df.groupby(['Cabin'])['Fare'].min().reset_index()
df_FareMean = df.groupby(['Cabin'])['Fare'].mean().reset_index()
df_FareMedian = df.groupby(['Cabin'])['Fare'].median().reset_index()
df_FareCount = df.groupby(['Cabin'])['Fare'].count().reset_index()
df_FareMode = df.groupby(['Cabin'])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
temp = pd.merge(df_FareMax, df_FareMin, how='left', on=['Cabin'])
temp = pd.merge(temp, df_FareMean, how='left', on=['Cabin'])
temp = pd.merge(temp, df_FareMedian, how='left', on=['Cabin'])
temp = pd.merge(temp, df_FareCount, how='left', on=['Cabin'])
temp = pd.merge(temp, df_FareMode, how='left', on=['Cabin'])
temp.columns = ['Cabin','FareMax', 'FareMin', 'FareMean', 'FareMedian', 'FareCount',  'FareMode']
temp

,Cabin,FareMax,FareMin,FareMean,FareMedian,FareCount,FareMode
0,A10,40.1250,40.1250,40.125000,40.12500,1,40.1250
1,A14,52.0000,52.0000,52.000000,52.00000,1,52.0000
2,A16,39.6000,39.6000,39.600000,39.60000,1,39.6000
3,A19,26.0000,26.0000,26.000000,26.00000,1,26.0000
4,A20,56.9292,56.9292,56.929200,56.92920,1,56.9292
5,A23,30.0000,30.0000,30.000000,30.00000,1,30.0000
6,A24,50.4958,50.4958,50.495800,50.49580,1,50.4958
7,A26,35.5000,35.5000,35.500000,35.50000,1,35.5000
8,A31,31.0000,31.0000,31.000000,31.00000,1,31.0000
9,A32,50.0000,50.0000,50.000000,50.00000,1,50.0000


In [6]:
df = copy.deepcopy(df)
df = pd.merge(df, temp, how='left', on=['Cabin'])
df = df.drop(['Cabin'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,FareMax,FareMin,FareMean,FareMedian,FareCount,FareMode
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,512.3292,0.0000,19.157325,10.5000,687,8.0500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,71.2833,71.2833,71.283300,71.2833,1,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,512.3292,0.0000,19.157325,10.5000,687,8.0500
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,53.1000,53.1000,53.100000,53.1000,2,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,512.3292,0.0000,19.157325,10.5000,687,8.0500


In [7]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

11 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'FareMax', 'FareMin', 'FareMean', 'FareMedian', 'FareCount', 'FareMode']



,Pclass,Age,SibSp,Parch,Fare,FareMax,FareMin,FareMean,FareMedian,FareCount,FareMode
0,3,22.0,1,0,7.2500,512.3292,0.0000,19.157325,10.5000,687,8.0500
1,1,38.0,1,0,71.2833,71.2833,71.2833,71.283300,71.2833,1,71.2833
2,3,26.0,0,0,7.9250,512.3292,0.0000,19.157325,10.5000,687,8.0500
3,1,35.0,1,0,53.1000,53.1000,53.1000,53.100000,53.1000,2,53.1000
4,3,35.0,0,0,8.0500,512.3292,0.0000,19.157325,10.5000,687,8.0500


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [8]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
# 沒有新特徵的 dataframe 稱為 df_minus
df_minus = df.drop(['FareMax', 'FareMin', 'FareMean', 'FareMedian','FareCount','FareMode'] , axis=1)
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.7038635542329971

In [9]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6903740601683743

> 沒有改善，反而下降，可能是Cabin特徵組內差異不大